In [1]:
words = open('names.txt','r').read().splitlines()

In [2]:
import torch

In [3]:
import torch.nn.functional as F

In [6]:
a = [[1,2]]
xs = torch.tensor(a)
xs

tensor([[1, 2]])

In [7]:
xenc = F.one_hot(xs, num_classes=27).float()
xenc

tensor([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [4]:
chars = ['.'] + sorted(list(set(''.join(words))))
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}

In [14]:
xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        # print(ch1, ch2)
        xs.append([ix1, ix2])
        ys.append(ix3)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.size(0)
print('number of examples: ',num)

number of examples:  196113


In [8]:
xs

tensor([[ 0,  5],
        [ 5, 13],
        [13, 13],
        ...,
        [26, 25],
        [25, 26],
        [26, 24]])

In [9]:
ys

tensor([13, 13,  1,  ..., 26, 24,  0])

In [16]:
g = torch.Generator().manual_seed(2147483647)
xenc = F.one_hot(xs, num_classes=27).float()
xenc = xenc.view(-1,2*27)
xenc.shape

torch.Size([196113, 54])

In [17]:
W = torch.randn((2*27, 27), generator=g, requires_grad=True)

In [20]:
for k in range(100):
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
    print(loss.item())
    
    W.grad = None
    loss.backward()

    W.data += -50 * W.grad

2.2679367065429688
2.267749309539795
2.2675657272338867
2.2673847675323486
2.267207384109497
2.267033100128174
2.2668616771698
2.266692876815796
2.2665271759033203
2.266364574432373
2.2662041187286377
2.2660470008850098
2.265892267227173
2.265739917755127
2.265590190887451
2.2654428482055664
2.265298366546631
2.26515531539917
2.265015125274658
2.2648768424987793
2.2647411823272705
2.2646074295043945
2.2644758224487305
2.26434588432312
2.264218330383301
2.2640926837921143
2.2639691829681396
2.2638471126556396
2.2637274265289307
2.2636091709136963
2.2634925842285156
2.2633779048919678
2.2632651329040527
2.2631540298461914
2.2630441188812256
2.2629363536834717
2.262829542160034
2.2627246379852295
2.2626211643218994
2.262519121170044
2.2624189853668213
2.262319564819336
2.2622218132019043
2.262125253677368
2.2620303630828857
2.261936664581299
2.2618443965911865
2.2617530822753906
2.2616629600524902
2.2615745067596436
2.2614872455596924
2.2614006996154785
2.2613155841827393
2.26123142242431

In [26]:
import random

In [33]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix1 = 0
    ix2 = random.randint(1,26)
    out.append(itos[ix2])

    while True:
        xenc = F.one_hot(torch.tensor([ix1, ix2]), num_classes=27).float()
        logits = xenc.view(-1,2*27) @ W
        counts = logits.exp()
        p = counts / counts.sum(1, keepdims=True)
        ix1 = ix2
        ix2 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix2])
        if ix2 == 0:
            break
    print(''.join(out))

counide.
cianasad.
ca.
colay.
ca.
